시간당 자전거 대여량을 예측하는 [Bike Sharing Demand](https://www.kaggle.com/c/bike-sharing-demand) 입니다. 워싱턴 D.C 소재의 자전거 대여 스타트업 [Capital Bikeshare](https://www.capitalbikeshare.com/)의 데이터를 활용하여, 특정 시간대에 얼마나 많은 사람들이 자전거를 대여하는지 예측하는 것이 목표입니다.

사람들이 자전거를 대여하는데는 많은 요소가 관여되어 있을 겁니다. 가령 **시간(새벽보다 낮에 많이 빌리겠죠)**, **날씨(비가 오면 자전거를 대여하지 않을 겁니다)**, **근무일(근무 시간에는 자전거를 대여하지 않겠죠)** 등. 이런 모든 요소를 조합하여 워싱턴 D.C의 자전거 교통량을 예측해주세요. 이번 경진대회에서는 기존까지 배웠던 프로그래밍 언어와 인공지능&머신러닝 능력 외에도, 자전거 렌탈 시장에 대한 약간의 전문지식, 그리고 일반인의 기초 상식을 총동원 할 수 있습니다.

In [1]:
# 필요한 패키지를 import 하는 구간
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [2]:
# 화면에 출력하는 데이터 프레임의 최대 row 수를 1000으로 설정합니다.
pd.set_option('display.max_rows', 1000)

# 화면에 출력하는 데이터 프레임의 최대 column 수를 1000으로 설정합니다.
pd.set_option('display.max_columns', 1000)

In [3]:
import matplotlib
from IPython.display import set_matplotlib_formats

# Mac OS를 이용하고 있기 떄문에 AppleGothic을 설정했습니다.
# 윈도우의 경우 다른 폰트를 사용해주셔야 합니다. 나눔폰트가 있는 경우 'NanumGothic' 없는 경우에는 'Malgun Gothic'을 사용해주세요.
matplotlib.rc('font', family='AppleGothic')
# matplotlib.rc('font', family='NanumGothic')

# 별도로, 폰트를 바꿀 경우 마이너스가 표시되지 않는 경우도 있는데 이를 막아주는 코드입니다.
matplotlib.rc('axes', unicode_minus=False)

# retina 옵션을 통해 해상도를 올립니다.
set_matplotlib_formats('retina')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 속성(컬럼) 설명(Column explain)

 * **datetime** - 시간. 연-월-일 시:분:초 로 표현합니다. (가령 2011-01-01 00:00:00은 2011년 1월 1일 0시 0분 0초)
 * **season** - 계절. 봄(1), 여름(2), 가을(3), 겨울(4) 순으로 표현합니다.
 * **holiday** - 공휴일. 1이면 공휴일이며, 0이면 공휴일이 아닙니다.
 * **workingday** - 근무일. 1이면 근무일이며, 0이면 근무일이 아닙니다.
 * **weather** - 날씨. 1 ~ 4 사이의 값을 가지며, 구체적으로는 다음과 같습니다.
   * 1: 아주 깨끗한 날씨입니다. 또는 아주 약간의 구름이 끼어있습니다.
   * 2: 약간의 안개와 구름이 끼어있는 날씨입니다.
   * 3: 약간의 눈, 비가 오거나 천둥이 칩니다.
   * 4: 아주 많은 비가 오거나 우박이 내립니다.
 * **temp** - 온도. 섭씨(Celsius)로 적혀있습니다.
 * **atemp** - 체감 온도. 마찬가지로 섭씨(Celsius)로 적혀있습니다.
 * **humidity** - 습도.
 * **windspeed** - 풍속.
 * **casual** - 비회원(non-registered)의 자전거 대여량.
 * **registered** - 회원(registered)의 자전거 대여량.
 * **count** - 총 자전거 대여랑. 비회원(casual) + 회원(registered)과 동일합니다.

### 데이터 셋 로드(Load Dataset)

In [4]:
# 데이터 셋을 로드하는 커널입니다
# 각각 data 폴더 내에 있는 train.csv, test.csv파일을 불러옵니다
# parse_dates : datetime 컬럼을 실제 날짜 처럼 연산하게 parsing 처리합니다
train = pd.read_csv("data/train.csv", parse_dates=["datetime"])
test = pd.read_csv("data/test.csv", parse_dates=["datetime"])

# 불러온 train데이터를 확인합니다.
print(train.shape)
train.head()

(10886, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [5]:
# 불러온 test 데이터를 확인합니다.
print(test.shape)
test.head()

(6493, 9)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


### 데이터 전처리(Data Preprocessing)

In [12]:
# train 데이터 전처리
# 먼저 datetime 데이터 처리 : 연, 월, 일, 시간, 분, 초 등으로 처리
train['datetime-year'] = train['datetime'].dt.year
train['datetime-month'] = train['datetime'].dt.month
train['datetime-day'] = train['datetime'].dt.day
train['datetime-hour'] = train['datetime'].dt.hour
train['datetime-minute'] = train['datetime'].dt.minute
train['datetime-second'] = train['datetime'].dt.second

print(train.shape)
train[["datetime-year", "datetime-month", "datetime-day", "datetime-hour", "datetime-minute",
       "datetime-second"]].head()

(10886, 20)


,datetime-year,datetime-month,datetime-day,datetime-hour,datetime-minute,datetime-second
0,2011,1,1,0,0,0
1,2011,1,1,1,0,0
2,2011,1,1,2,0,0
3,2011,1,1,3,0,0
4,2011,1,1,4,0,0


In [13]:
# test 데이터 전처리
# train과 동일하게 진행합니다.
test['datetime-year'] = test['datetime'].dt.year
test['datetime-month'] = test['datetime'].dt.month
test['datetime-day'] = test['datetime'].dt.day
test['datetime-hour'] = test['datetime'].dt.hour
test['datetime-minute'] = test['datetime'].dt.minute
test['datetime-second'] = test['datetime'].dt.second

print(test.shape)
test[["datetime-year", "datetime-month", "datetime-day", "datetime-hour", "datetime-minute",
       "datetime-second"]].head()

(6493, 17)


,datetime-year,datetime-month,datetime-day,datetime-hour,datetime-minute,datetime-second
0,2011,1,20,0,0,0
1,2011,1,20,1,0,0
2,2011,1,20,2,0,0
3,2011,1,20,3,0,0
4,2011,1,20,4,0,0


In [8]:
# 요일의 정보를 추가합니다.
# train, test 모두 다 컬럼을 추가합니다.
train["datetime-dayofweek"] = train["datetime"].dt.dayofweek
test["datetime-dayofweek"] = test["datetime"].dt.dayofweek

print(train.shape)
train.head()

print(test.shape)
test.head()

(10886, 19)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,datetime-year,datetime-month,datetime-day,datetime-hour,datetime-minute,datetime-second,datetime-dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0,0,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1,0,0,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2,0,0,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,1,3,0,0,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,1,4,0,0,5


(6493, 16)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,datetime-year,datetime-month,datetime-day,datetime-hour,datetime-minute,datetime-second,datetime-dayofweek
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,0,0,0,3
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,1,0,0,3
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,2,0,0,3
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,0,0,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,4,0,0,3


In [14]:
# dayofweek가 정수값으로 나타내어져 있어 실제 데이터분석에 다소 불편한 감이 있습니다
# 따라서 사람이 알아보기 쉬운 Mon, Tue, ... , Sun 값으로 바꾸는 과정입니다.
# train 데이터
train.loc[train["datetime-dayofweek"] == 0, "datetime-dayofweek(hum)"] = "Monday"
train.loc[train["datetime-dayofweek"] == 1, "datetime-dayofweek(hum)"] = "Tuesday"
train.loc[train["datetime-dayofweek"] == 2, "datetime-dayofweek(hum)"] = "Wednesday"
train.loc[train["datetime-dayofweek"] == 3, "datetime-dayofweek(hum)"] = "Thursday"
train.loc[train["datetime-dayofweek"] == 4, "datetime-dayofweek(hum)"] = "Friday"
train.loc[train["datetime-dayofweek"] == 5, "datetime-dayofweek(hum)"] = "Saturday"
train.loc[train["datetime-dayofweek"] == 6, "datetime-dayofweek(hum)"] = "Sunday"

print(train.shape)
train[["datetime-year", "datetime-month", "datetime-day", "datetime-dayofweek(hum)",
       "datetime-hour", "datetime-minute", "datetime-second"]].head()

(10886, 20)


,datetime-year,datetime-month,datetime-day,datetime-dayofweek(hum),datetime-hour,datetime-minute,datetime-second
0,2011,1,1,Saturday,0,0,0
1,2011,1,1,Saturday,1,0,0
2,2011,1,1,Saturday,2,0,0
3,2011,1,1,Saturday,3,0,0
4,2011,1,1,Saturday,4,0,0


In [15]:
# test 데이터
test.loc[train["datetime-dayofweek"] == 0, "datetime-dayofweek(hum)"] = "Monday"
test.loc[train["datetime-dayofweek"] == 1, "datetime-dayofweek(hum)"] = "Tuesday"
test.loc[train["datetime-dayofweek"] == 2, "datetime-dayofweek(hum)"] = "Wednesday"
test.loc[train["datetime-dayofweek"] == 3, "datetime-dayofweek(hum)"] = "Thursday"
test.loc[train["datetime-dayofweek"] == 4, "datetime-dayofweek(hum)"] = "Friday"
test.loc[train["datetime-dayofweek"] == 5, "datetime-dayofweek(hum)"] = "Saturday"
test.loc[train["datetime-dayofweek"] == 6, "datetime-dayofweek(hum)"] = "Sunday"

print(test.shape)
test[["datetime-year", "datetime-month", "datetime-day", "datetime-dayofweek(hum)",
       "datetime-hour", "datetime-minute", "datetime-second"]].head()

(6493, 17)


,datetime-year,datetime-month,datetime-day,datetime-dayofweek(hum),datetime-hour,datetime-minute,datetime-second
0,2011,1,20,Saturday,0,0,0
1,2011,1,20,Saturday,1,0,0
2,2011,1,20,Saturday,2,0,0
3,2011,1,20,Saturday,3,0,0
4,2011,1,20,Saturday,4,0,0


### 탐험적 데이터 분석(EDA)

### 머신러닝 파트(Machine Learning Part)